# <span style="font-width:bold; font-size: 3rem; color:#2656a3;">**Data Engineering and Machine Learning Operations in Business** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 03: Training Pipeline</span>

## 🗒️ This notebook is divided into the following sections:
1. Feature selection.
2. Feature transformations.
3. Training datasets creation.
4. Loading the training data.
5. Train the model.
6. Register model to Hopsworks model registry.

## <span style='color:#2656a3'> ⚙️ Import of libraries and packages

In [1]:
!pip install tensorflow --quiet

In [2]:
import inspect 
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

2024-04-16 16:06:19.917866: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## <span style="color:#2656a3;"> 📡 Connecting to Hopsworks Feature Store

In [3]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/550040
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
# Retrieve feature groups
electricity_fg = fs.get_feature_group(
    name='electricity_prices',
    version=1,
)

weather_fg = fs.get_feature_group(
    name='weather_measurements',
    version=1,
)

danish_holidays_fg = fs.get_feature_group(
    name='danish_holidays',
    version=1,
)

In [5]:
# Select features for training data
selected_features = electricity_fg.select_all()\
    .join(
    weather_fg\
        .select_except(["timestamp"])
    )\
    .join(
        danish_holidays_fg.select_all()
    )

In [6]:
transformation_functions = {
        "SpotPriceDKK_KWH": fs.get_transformation_function(name="min_max_scaler"), 
        "temperature_2m": fs.get_transformation_function(name="min_max_scaler"), 
        "relative_humidity_2m": fs.get_transformation_function(name="min_max_scaler"), 
        "precipitation": fs.get_transformation_function(name="min_max_scaler"), 
        "rain": fs.get_transformation_function(name="min_max_scaler"), 
        "snowfall": fs.get_transformation_function(name="min_max_scaler"), 
        "weather_code": fs.get_transformation_function(name="min_max_scaler"), 
        "cloud_cover": fs.get_transformation_function(name="min_max_scaler"), 
        "wind_speed_10m": fs.get_transformation_function(name="min_max_scaler"),
        "wind_gusts_10m": fs.get_transformation_function(name="min_max_scaler")
    }

In [7]:
feature_view = fs.get_or_create_feature_view(
    name='electricity_feature_view',
    version=1,
    labels=[], # you will define our 'y' later manualy
    transformation_functions=transformation_functions,
    query=selected_features,
)

## Training Dataset Creation


### Dataset with train, test and validation splits

In [ ]:
# since you didn't specify 'labels' in feature view creation, it will return None for Y.
X_train, X_val, X_test, _, _, _ = feature_view.train_validation_test_split(
    train_start="2021-01-01",
    train_end="2022-02-28",
    validation_start="2022-03-01",
    validation_end="2022-05-31",
    test_start="2022-06-01",
    test_end="2022-09-09",
    description='Electricity price prediction dataset',
)